In [2]:
from scipy import stats
import pandas as pd
import numpy as np

import pandas_datareader.data as pdr
import yfinance as yf
yf.pdr_override()
from datetime import datetime

import fundamentalanalysis as fa
api_key = "56bd4a3ac58b13bd6c93c37ff7f06dac"

import scipy.optimize as sco

import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
available = fa.available_companies(api_key)
available

,name,price,exchange,exchangeShortName,type
symbol,,,,,
NPOF.ME,FIZIKA Scientific and Production Association O...,11850.0000,MCX,MCX,stock
KZMS.ME,The Open Joint Stock Company Krasnokamsk Metal...,226.0000,MCX,MCX,stock
AFAXX,American Funds U.S. Government Money Market Fu...,1.0000,Nasdaq,NASDAQ,trust
IFT.NZ,Infratil Limited,9.7400,NZSE,NZE,stock
AIR.NZ,Air New Zealand Limited,0.8000,NZSE,NZE,stock
...,...,...,...,...,...
LLL,JX Luxventure Limited,0.8391,NASDAQ Capital Market,NASDAQ,stock
TPGY-UN,TPG Pace Beneficial Finance Corp.,10.0500,New York Stock Exchange,NYSE,stock
WBIT,WBI BullBear Trend Switch US 3000 Total Return...,18.1100,New York Stock Exchange Arca,AMEX,etf


In [2]:
available[available['exchange'] == "Nasdaq"]['type'].value_counts()

NameError: name 'available' is not defined

In [4]:
# Get S&P 500 ticketer symbol
payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = pd.DataFrame(payload[0])
stock_symbols = df['Symbol'].values.tolist()

In [100]:
# Create a DF for the Key Metrics
final_key_metrics = pd.DataFrame(columns=['years', 'value', 'prev_return', 'exp_return', 'Sector', 'Name',
       'period', 'revenuePerShare', 'netIncomePerShare',
       'operatingCashFlowPerShare', 'freeCashFlowPerShare', 'cashPerShare',
       'bookValuePerShare', 'tangibleBookValuePerShare',
       'shareholdersEquityPerShare', 'interestDebtPerShare', 'marketCap',
       'enterpriseValue', 'peRatio', 'priceToSalesRatio', 'pocfratio',
       'pfcfRatio', 'pbRatio', 'ptbRatio', 'evToSales',
       'enterpriseValueOverEBITDA', 'evToOperatingCashFlow',
       'evToFreeCashFlow', 'earningsYield', 'freeCashFlowYield',
       'debtToEquity', 'debtToAssets', 'netDebtToEBITDA', 'currentRatio',
       'interestCoverage', 'incomeQuality', 'dividendYield', 'payoutRatio',
       'salesGeneralAndAdministrativeToRevenue',
       'researchAndDdevelopementToRevenue', 'intangiblesToTotalAssets',
       'capexToOperatingCashFlow', 'capexToRevenue', 'capexToDepreciation',
       'stockBasedCompensationToRevenue', 'grahamNumber', 'roic',
       'returnOnTangibleAssets', 'grahamNetNet', 'workingCapital',
       'tangibleAssetValue', 'netCurrentAssetValue', 'investedCapital',
       'averageReceivables', 'averagePayables', 'averageInventory',
       'daysSalesOutstanding', 'daysPayablesOutstanding',
       'daysOfInventoryOnHand', 'receivablesTurnover', 'payablesTurnover',
       'inventoryTurnover', 'roe', 'capexPerShare'])

In [21]:
# final DF for financial ratios
fina_financial_ratios = pd.DataFrame(columns=['years', 'value', 'prev_return', 'exp_return', 'Sector', 'Name',
       'period', 'currentRatio', 'quickRatio', 'cashRatio',
       'daysOfSalesOutstanding', 'daysOfInventoryOutstanding',
       'operatingCycle', 'daysOfPayablesOutstanding', 'cashConversionCycle',
       'grossProfitMargin', 'operatingProfitMargin', 'pretaxProfitMargin',
       'netProfitMargin', 'effectiveTaxRate', 'returnOnAssets',
       'returnOnEquity', 'returnOnCapitalEmployed', 'netIncomePerEBT',
       'ebtPerEbit', 'ebitPerRevenue', 'debtRatio', 'debtEquityRatio',
       'longTermDebtToCapitalization', 'totalDebtToCapitalization',
       'interestCoverage', 'cashFlowToDebtRatio', 'companyEquityMultiplier',
       'receivablesTurnover', 'payablesTurnover', 'inventoryTurnover',
       'fixedAssetTurnover', 'assetTurnover', 'operatingCashFlowPerShare',
       'freeCashFlowPerShare', 'cashPerShare', 'payoutRatio',
       'operatingCashFlowSalesRatio', 'freeCashFlowOperatingCashFlowRatio',
       'cashFlowCoverageRatios', 'shortTermCoverageRatios',
       'capitalExpenditureCoverageRatio', 'dividendPaidAndCapexCoverageRatio',
       'dividendPayoutRatio', 'priceBookValueRatio', 'priceToBookRatio',
       'priceToSalesRatio', 'priceEarningsRatio', 'priceToFreeCashFlowsRatio',
       'priceToOperatingCashFlowsRatio', 'priceCashFlowRatio',
       'priceEarningsToGrowthRatio', 'priceSalesRatio', 'dividendYield',
       'enterpriseValueMultiple', 'priceFairValue'])

In [102]:
# final Df for financial growth
final_gowth = pd.DataFrame(columns=['years', 'value', 'prev_return', 'exp_return', 'Sector', 'Name',
       'period', 'revenueGrowth', 'grossProfitGrowth', 'ebitgrowth',
       'operatingIncomeGrowth', 'netIncomeGrowth', 'epsgrowth',
       'epsdilutedGrowth', 'weightedAverageSharesGrowth',
       'weightedAverageSharesDilutedGrowth', 'dividendsperShareGrowth',
       'operatingCashFlowGrowth', 'freeCashFlowGrowth',
       'tenYRevenueGrowthPerShare', 'fiveYRevenueGrowthPerShare',
       'threeYRevenueGrowthPerShare', 'tenYOperatingCFGrowthPerShare',
       'fiveYOperatingCFGrowthPerShare', 'threeYOperatingCFGrowthPerShare',
       'tenYNetIncomeGrowthPerShare', 'fiveYNetIncomeGrowthPerShare',
       'threeYNetIncomeGrowthPerShare', 'tenYShareholdersEquityGrowthPerShare',
       'fiveYShareholdersEquityGrowthPerShare',
       'threeYShareholdersEquityGrowthPerShare',
       'tenYDividendperShareGrowthPerShare',
       'fiveYDividendperShareGrowthPerShare',
       'threeYDividendperShareGrowthPerShare', 'receivablesGrowth',
       'inventoryGrowth', 'assetGrowth', 'bookValueperShareGrowth',
       'debtGrowth', 'rdexpenseGrowth', 'sgaexpensesGrowth'])

In [35]:
stock_symbols.index(ticker)
df = fina_financial_ratios.copy()

In [32]:

for ticker in stock_symbols[95:250]:
    stock_values = {
    'years' : ['2017','2018', '2019', '2020', '2021', '2022'],
    'value': [],
    }

    for x in stock_values['years']:
        values = pdr.DataReader(ticker,start=x+'-9-1', end=x+'-9-30')['Adj Close']
        stock_values['value'].append(values.mean())


    # Create a DF with the year returns and next year returns
    stock_values_df = pd.DataFrame(stock_values)
    stock_values_df['prev_return'] = stock_values_df['value'].pct_change()
    stock_values_df = stock_values_df.iloc[1:,:]
    stock_values_df['exp_return'] = stock_values_df['prev_return'].shift(-1)
    stock_values_df

    # Add the Sector and Name to the DF
    tickerdata = yf.Ticker(ticker) #the tickersymbol for Tesla
    stock_values_df['Sector'] = tickerdata.info['sector']
    stock_values_df['Name'] = ticker
    stock_values_df['years'].sort_values(ascending=False)
    stock_values_df

    # # Collect Data for the Key Metrics
    # key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
    # key_metrics_annually = np.transpose(key_metrics_annually).reset_index()
    # key_metrics_annually = pd.merge(stock_values_df, key_metrics_annually, left_on='years', right_on='index', how='left').drop('index', axis=1)
    # final_key_metrics = pd.concat([final_key_metrics,key_metrics_annually])

    # Create Financial Ratios
    financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
    financial_ratios_annually = np.transpose(financial_ratios_annually).reset_index()
    financial_ratios_annually = pd.merge(stock_values_df, financial_ratios_annually, left_on='years', right_on='index', how='left').drop('index', axis=1)
    fina_financial_ratios = pd.concat([fina_financial_ratios, financial_ratios_annually])

    # # Create Statment Growth
    # financial_statement_growth = fa.financial_statement_growth(ticker, api_key, period="annual")
    # financial_statement_growth = np.transpose(financial_statement_growth).reset_index()
    # financial_statement_growth = pd.merge(stock_values_df, financial_statement_growth, left_on='years', right_on='index', how='left').drop('index', axis=1)
    # final_gowth = pd.concat([final_gowth, financial_statement_growth])



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

ERROR 
1 Failed download:
ERROR ['CDAY']: Exception("CDAY: Data doesn't exist for startDate = 1504238400, endDate = 1506744000")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

ERROR 
1 Failed download:
ERROR ['CEG']: Exception("CEG: Data doesn't exist for startDate = 1504238400, endDate = 1506744000")



[*********************100%***********************]  1 of 1 completed

...


[*********************100%***********************]  1 of 1 completed

ERROR 
1 Failed download:
ERROR ['GEHC']: Exception("GEHC: Data doesn't exist for startDate = 1630468800, endDate = 1632974400")



[*********************100%***********************]  1 of 1 completed

ERROR 
1 Failed download:
ERROR ['GEHC']: Exception("GEHC: Data doesn't exist for startDate = 1662004800, endDate = 1664510400")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

In [168]:
# Test
# final_key_metrics = final_key_metrics[final_key_metrics['Name'].isin(stock_symbols[50:65])]
# fina_financial_ratios = fina_financial_ratios[fina_financial_ratios['Name'].isin(stock_symbols[50:65])]
# final_gowth = final_gowth[final_gowth['Name'].isin(stock_symbols[50:65])]

# final_key_metrics = final_key_metrics[final_key_metrics['Name'] != 'CARR']
# fina_financial_ratios = fina_financial_ratios[fina_financial_ratios['Name'] != 'CARR']
# final_gowth = final_gowth[final_gowth['Name'] != 'CARR']

# final_key_metrics = final_key_metrics[~final_key_metrics['prev_return'].isna()]
# fina_financial_ratios = fina_financial_ratios[~fina_financial_ratios['prev_return'].isna()]
# final_gowth = final_gowth[~final_gowth['prev_return'].isna()]

final_gowth["Name"].unique()

array(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP',
       'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK',
       'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA',
       'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO',
       'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI',
       'BAX', 'BDX', 'WRB', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK',
       'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO',
       'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX',
       'CCL', 'CTLT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY',
       'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI',
       'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO',
       'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO',
     

In [36]:
# Concat from old DF 
# final_key_metrics = pd.concat([pd.read_csv('key_metrics.csv'), final_key_metrics]) 
fina_financial_ratios = pd.concat([pd.read_csv('ratios.csv'), fina_financial_ratios]) 
# final_gowth = pd.concat([pd.read_csv('growth.csv'), final_gowth])

In [44]:
# Saving
# final_key_metrics.to_csv('key_metrics.csv', index=False)
fina_financial_ratios.to_csv('ratios.csv', index=False)
# final_gowth.to_csv('growth.csv', index=False)

 ********* INDIVIDUAL TESTS *********

In [89]:
# Collect Stock Values 
ticker = 'ATVI'

stock_values = {
    'years' : ['2017','2018', '2019', '2020', '2021', '2022'],
    'value': [],
}

for x in stock_values['years']:
    values = pdr.DataReader(ticker,start=x+'-9-1', end=x+'-9-30')['Adj Close']
    stock_values['value'].append(values.mean())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [90]:
# Create a DF with the year returns and next year returns 
stock_values_df = pd.DataFrame(stock_values)
stock_values_df['prev_return'] = stock_values_df['value'].pct_change()
stock_values_df = stock_values_df.iloc[1:,:]
stock_values_df['exp_return'] = stock_values_df['prev_return'].shift(-1)
stock_values_df

,years,value,prev_return,exp_return
1,2018,76.662574,0.225978,-0.300349
2,2019,53.637035,-0.300349,0.481304
3,2020,79.452738,0.481304,-0.026938
4,2021,77.312448,-0.026938,-0.011408
5,2022,76.430500,-0.011408,NaN


In [91]:
# Add the Sector and Name to the DF 
tickerdata = yf.Ticker(ticker) #the tickersymbol for Tesla
stock_values_df['Sector'] = tickerdata.info['sector']
stock_values_df['Name'] = ticker
stock_values_df['years'].sort_values(ascending=False)
stock_values_df

,years,value,prev_return,exp_return,Sector,Name
1,2018,76.662574,0.225978,-0.300349,Communication Services,ATVI
2,2019,53.637035,-0.300349,0.481304,Communication Services,ATVI
3,2020,79.452738,0.481304,-0.026938,Communication Services,ATVI
4,2021,77.312448,-0.026938,-0.011408,Communication Services,ATVI
5,2022,76.430500,-0.011408,NaN,Communication Services,ATVI


In [92]:
final_key_metrics[final_key_metrics['Name'] == "ATVI"]

,years,value,prev_return,exp_return,Sector,Name,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
25,2018,76.662575,0.225978,0.000000,Communication Services,ATVI,FY,9.842520,2.379265,2.349081,...,288000000,44500000,50.370000,36.688518,6.235598,7.246377,9.948617,58.534884,0.159637,-0.171916
26,2019,NaN,0.000000,0.036395,Communication Services,ATVI,FY,8.460235,1.959583,2.387223,...,272500000,37500000,47.699183,50.897803,5.577841,7.652123,7.171233,65.437500,0.117376,-0.151239
27,2020,79.452740,0.036395,-0.026938,Communication Services,ATVI,FY,10.487678,2.849546,2.920882,...,293500000,16000000,47.487015,47.643805,0.000000,7.686312,7.661017,0.000000,0.146106,-0.101167
28,2021,77.312448,-0.026938,-0.011408,Communication Services,ATVI,FY,11.329472,3.473616,3.106821,...,290000000,0,40.302170,44.896418,0.000000,9.056584,8.129825,0.000000,0.153361,-0.102960
29,2022,76.430500,-0.011408,NaN,Communication Services,ATVI,FY,9.626598,1.934783,2.838875,...,304500000,0,58.376727,53.222322,0.000000,6.252492,6.858025,0.000000,0.078626,-0.116368


In [98]:
# Collect Data for the Key Metrics
key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
key_metrics_annually = np.transpose(key_metrics_annually).reset_index()
key_metrics_annually = pd.merge(stock_values_df, key_metrics_annually, left_on='years', right_on='index', how='left').drop('index', axis=1)
final_key_metrics = pd.concat([final_key_metrics,key_metrics_annually])

,years,value,prev_return,exp_return,Sector,Name,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,2018,176.984499,0.036314,-0.189466,Industrials,MMM,FY,55.675446,9.08921,10.941376,...,2105500000,4200000000,57.069892,49.579787,95.527515,6.395667,7.361871,3.820889,0.546039,-2.679694
1,2019,143.451944,-0.189466,0.029663,Industrials,MMM,FY,55.694974,7.920277,12.253033,...,2247000000,4250000000,54.416075,47.456816,88.054972,6.707577,7.691203,4.145138,0.451313,-2.944541
2,2020,147.707197,0.029663,0.154438,Industrials,MMM,FY,55.720222,9.32133,14.046053,...,2394500000,4186500000,53.359589,56.294188,93.178862,6.840383,6.483795,3.917197,0.416364,-2.598684
3,2021,170.518749,0.154438,-0.338100,Industrials,MMM,FY,61.062176,10.226252,12.873921,...,2777500000,4612000000,48.109178,58.143655,96.808992,7.58691,6.277555,3.770311,0.391678,-2.768566
4,2022,112.866316,-0.338100,NaN,Industrials,MMM,FY,60.475265,10.206714,9.878092,...,3088500000,5178500000,48.326857,60.409474,101.954035,7.552736,6.042099,3.580045,0.391131,-3.090106


In [99]:
# Create Financial Ratios
financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
financial_ratios_annually = np.transpose(financial_ratios_annually).reset_index()
financial_ratios_annually = pd.merge(stock_values_df, financial_ratios_annually, left_on='years', right_on='index', how='left').drop('index', axis=1)
fina_financial_ratios = pd.concat([fina_financial_ratios, financial_ratios_annually])

Index(['years', 'value', 'prev_return', 'exp_return', 'Sector', 'Name',
       'period', 'currentRatio', 'quickRatio', 'cashRatio',
       'daysOfSalesOutstanding', 'daysOfInventoryOutstanding',
       'operatingCycle', 'daysOfPayablesOutstanding', 'cashConversionCycle',
       'grossProfitMargin', 'operatingProfitMargin', 'pretaxProfitMargin',
       'netProfitMargin', 'effectiveTaxRate', 'returnOnAssets',
       'returnOnEquity', 'returnOnCapitalEmployed', 'netIncomePerEBT',
       'ebtPerEbit', 'ebitPerRevenue', 'debtRatio', 'debtEquityRatio',
       'longTermDebtToCapitalization', 'totalDebtToCapitalization',
       'interestCoverage', 'cashFlowToDebtRatio', 'companyEquityMultiplier',
       'receivablesTurnover', 'payablesTurnover', 'inventoryTurnover',
       'fixedAssetTurnover', 'assetTurnover', 'operatingCashFlowPerShare',
       'freeCashFlowPerShare', 'cashPerShare', 'payoutRatio',
       'operatingCashFlowSalesRatio', 'freeCashFlowOperatingCashFlowRatio',
       'cashFlowCo

In [126]:
# Create Statment Growth
financial_statement_growth = fa.financial_statement_growth(ticker, api_key, period="annual")
financial_statement_growth = np.transpose(financial_statement_growth).reset_index()
financial_statement_growth = pd.merge(stock_values_df, financial_statement_growth, left_on='years', right_on='index', how='left').drop('index', axis=1)
final_gowth = pd.concat([final_gowth, financial_statement_growth])

In [127]:
final_gowth

,years,value,prev_return,exp_return,Sector,Name,period,revenueGrowth,grossProfitGrowth,ebitgrowth,...,tenYDividendperShareGrowthPerShare,fiveYDividendperShareGrowthPerShare,threeYDividendperShareGrowthPerShare,receivablesGrowth,inventoryGrowth,assetGrowth,bookValueperShareGrowth,debtGrowth,rdexpenseGrowth,sgaexpensesGrowth
0,2018,176.984504,0.036314,-0.189466,Industrials,MMM,FY,0.035,0.027274,-0.078389,...,1.713605,1.138588,0.325378,0.028302,0.0823,-0.039145,-0.139859,0.048247,-0.015676,0.145618
1,2019,143.451937,-0.189466,0.029663,Industrials,MMM,FY,-0.019197,-0.067338,-0.396004,...,1.813243,0.683633,0.297681,-0.064806,-0.053138,0.223534,0.054289,0.4061,0.049423,-0.06641
2,2020,147.707192,0.029663,0.154438,Industrials,MMM,FY,0.001494,0.0386,0.027337,...,1.790877,0.432859,0.250348,-0.01795,0.025399,0.060122,0.275683,-0.073395,-0.017268,-0.014227
3,2021,170.518745,0.154438,-0.338100,Industrials,MMM,FY,0.098527,0.062969,0.219589,...,1.691268,0.333758,0.088667,-0.009564,0.175985,-0.005745,0.166224,-0.074799,0.061768,0.038678
4,2022,112.866316,-0.338100,NaN,Industrials,MMM,FY,-0.031848,-0.094384,-1.036304,...,1.526177,0.268818,0.035728,-0.027468,0.077633,-0.013108,-0.000513,-0.080903,-0.066199,0.257329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018,127.565162,0.016812,0.109224,Industrials,ITW,FY,0.031717,0.026478,0.025758,...,1.906428,1.835418,0.663594,-0.007095,0.080328,-0.113826,-0.271016,-0.113833,0,-0.00375
1,2019,141.498353,0.109224,0.297888,Industrials,ITW,FY,-0.044624,-0.03926,-0.050781,...,2.291906,1.304221,0.763507,-0.120757,-0.116844,0.013315,-0.038199,0.05122,0,-0.012547
2,2020,183.649007,0.297888,0.146008,Industrials,ITW,FY,-0.108796,-0.122087,-0.152851,...,2.425219,1.157585,0.591245,0.018285,0.021478,0.036103,0.074443,0.046919,0,-0.083863
3,2021,210.463209,0.146008,-0.089228,Industrials,ITW,FY,0.149594,0.147528,0.206454,...,2.354114,1.007618,0.383803,0.13328,0.424727,0.029785,0.146089,-0.053558,0,0.089228


****** FINISH INDIVIDUAL TEST *********

In [46]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


In [94]:
# Read DF 
final_key_metrics = pd.read_csv('key_metrics.csv')
fina_financial_ratios = pd.read_csv('ratios.csv')
final_gowth = pd.read_csv('growth.csv')

245

In [95]:
# Test the data 
final_key_metrics = final_key_metrics[~final_key_metrics['period'].isna()]
fina_financial_ratios = fina_financial_ratios[~fina_financial_ratios['period'].isna()]
final_gowth = final_gowth[~final_gowth['period'].isna()]

In [128]:
final_key_metrics

,years,value,prev_return,exp_return,Sector,Name,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,2018,176.984504,0.036314,-0.189466,Industrials,MMM,FY,55.675446,9.089210,10.941376,...,2.105500e+09,4.200000e+09,57.069892,49.579787,95.527515,6.395667,7.361871,3.820889,0.546039,-2.679694
1,2019,143.451937,-0.189466,0.029663,Industrials,MMM,FY,55.694974,7.920277,12.253033,...,2.247000e+09,4.250000e+09,54.416075,47.456816,88.054972,6.707577,7.691203,4.145138,0.451313,-2.944541
2,2020,147.707192,0.029663,0.154438,Industrials,MMM,FY,55.720222,9.321330,14.046053,...,2.394500e+09,4.186500e+09,53.359589,56.294188,93.178862,6.840383,6.483795,3.917197,0.416364,-2.598684
3,2021,170.518745,0.154438,-0.338100,Industrials,MMM,FY,61.062176,10.226252,12.873921,...,2.777500e+09,4.612000e+09,48.109178,58.143655,96.808992,7.586910,6.277555,3.770311,0.391678,-2.768566
4,2022,112.866316,-0.338100,NaN,Industrials,MMM,FY,60.475265,10.206714,9.878092,...,3.088500e+09,5.178500e+09,48.326857,60.409474,101.954035,7.552736,6.042099,3.580045,0.391131,-3.090106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,2018,80.673477,0.039665,0.216202,Utilities,DTE,FY,78.519337,6.187845,14.806630,...,1.250000e+09,7.950000e+08,48.719744,43.851474,26.759628,7.491829,8.323552,13.639951,0.109407,-14.988950
760,2019,98.115221,0.216202,-0.078355,Utilities,DTE,FY,68.481081,6.318919,14.318919,...,1.202500e+09,7.850000e+08,54.365380,42.366775,29.885113,6.713831,8.615242,12.213439,0.100154,-16.200000
761,2020,90.427387,-0.078355,0.218040,Utilities,DTE,FY,63.093264,7.088083,19.155440,...,1.052500e+09,7.375000e+08,53.714380,45.234855,31.475370,6.795201,8.068999,11.596369,0.110101,-19.984456
762,2021,110.144160,0.218040,0.141745,Utilities,DTE,FY,77.533679,4.699482,15.891192,...,1.221500e+09,7.870000e+08,44.637129,44.385105,26.932405,8.177049,8.223479,13.552448,0.104193,-19.544041


Key Metrics

In [96]:
# Separate the data by years
final_key_metrics['years'] = final_key_metrics['years'].map(lambda x: int(x))
train_metrics = final_key_metrics[final_key_metrics['years'].isin([2018, 2019, 2020])]
test_metrics = final_key_metrics[final_key_metrics['years'].isin([2021])]

# Remove Years and Period
train_metrics = train_metrics.drop(['years', 'period', 'Name'], axis=1)
test_metrics = test_metrics.drop(['years', 'period', 'Name'], axis=1)

le = preprocessing.LabelEncoder()

# Transform categorical variables
train_metrics['Sector'] = le.fit_transform(train_metrics['Sector'])
test_metrics['Sector'] = le.fit_transform(test_metrics['Sector'])

# Split Data 
train_Y_metric = train_metrics['exp_return'].astype(float)
train_X_metric = train_metrics.drop('exp_return', axis=1).astype(float)

test_Y_metric = test_metrics['exp_return'].astype(float)
test_X_metric = test_metrics.drop('exp_return', axis=1).astype(float)

# Random Forest
ran_f = RandomForestRegressor()

# Fit the data 
ran_f.fit(train_X_metric, train_Y_metric)

# Predict
prediction = ran_f.predict(test_X_metric)





In [97]:
# Test
mae = mean_absolute_error(test_Y_metric*100, prediction*100)
mse = mean_squared_error(test_Y_metric*100, prediction*100, squared=False)

# Print
print("MAE: ", mae)
print("MSE: ", mse)

MAE:  39.42522260176216
MSE:  48.66650035811645


In [98]:
importances = ran_f.feature_importances_

# To make it more understandable, let's create a DataFrame and print it
feature_importances = pd.DataFrame({'feature': train_X_metric.columns, 'importance': importances})
print(feature_importances.sort_values(by='importance', ascending=False))

                                   feature  importance
1                              prev_return    0.093554
21               enterpriseValueOverEBITDA    0.058724
28                         netDebtToEBITDA    0.034646
22                   evToOperatingCashFlow    0.034157
43                  returnOnTangibleAssets    0.032874
24                           earningsYield    0.031899
0                                    value    0.031666
7                             cashPerShare    0.028775
40         stockBasedCompensationToRevenue    0.028055
59                           capexPerShare    0.022406
47                    netCurrentAssetValue    0.020312
49                      averageReceivables    0.019671
23                        evToFreeCashFlow    0.018707
17                               pfcfRatio    0.018092
25                       freeCashFlowYield    0.017751
27                            debtToAssets    0.017728
38                          capexToRevenue    0.017552
16        

In [101]:
df = pd.DataFrame({'feature': final_key_metrics[final_key_metrics['years'].isin([2021])].Name, 'importance': prediction})
print(df.sort_values('importance', ascending=False))



ValueError: array length 250 does not match index length 156

# Ratios

In [124]:
# Separate the data by years
fina_financial_ratios['years'] = fina_financial_ratios['years'].map(lambda x: int(x))
fina_financial_ratios.drop_duplicates(subset=['years', 'Name'], inplace=True)
train_metrics = fina_financial_ratios[fina_financial_ratios['years'].isin([2018, 2019, 2020])]
test_metrics = fina_financial_ratios[fina_financial_ratios['years'].isin([2021])]

# Remove Years and Period
train_metrics = train_metrics.drop(['years', 'period', 'Name', 'operatingCycle', 'cashConversionCycle'], axis=1)
test_metrics = test_metrics.drop(['years', 'period', 'Name', 'operatingCycle', 'cashConversionCycle'], axis=1)

le = preprocessing.LabelEncoder()

# Transform categorical variables
train_metrics['Sector'] = le.fit_transform(train_metrics['Sector'])
test_metrics['Sector'] = le.fit_transform(test_metrics['Sector'])

# Split Data 
train_Y_metric = train_metrics['exp_return'].astype(float)
train_X_metric = train_metrics.drop('exp_return', axis=1).astype(float)

test_Y_metric = test_metrics['exp_return'].astype(float)
test_X_metric = test_metrics.drop('exp_return', axis=1).astype(float)

# Random Forest
ran_f = RandomForestRegressor()

# Fit the data 
ran_f.fit(train_X_metric, train_Y_metric)

# Predict
prediction = ran_f.predict(test_X_metric)





,years,value,prev_return,exp_return,Sector,Name,period,currentRatio,quickRatio,cashRatio,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
64,2018,80.673477,0.039665,0.216202,Utilities,DTE,FY,0.734565,0.443443,0.015998,...,1.404750,17.825268,-604.978788,7.449366,7.449366,-8.046835,1.404750,0.031055,11.766736,1.950210
65,2019,98.115221,0.216202,-0.078355,Utilities,DTE,FY,0.772079,0.495372,0.023267,...,1.896436,20.552566,-69.040086,9.069819,9.069819,9.072490,1.896436,0.028802,13.611714,2.058426
66,2020,90.427387,-0.078355,0.218040,Utilities,DTE,FY,1.299889,0.856931,0.191007,...,1.924294,17.128750,-146.450813,6.338147,6.338147,1.405892,1.924294,0.032434,12.011888,1.885886
67,2021,110.144160,0.218040,0.141745,Utilities,DTE,FY,0.522691,0.292783,0.004412,...,1.541782,25.436847,-32.725135,7.522406,7.522406,-0.748329,1.541782,0.034285,13.664533,2.650341
68,2022,125.756582,0.141745,NaN,Utilities,DTE,FY,0.808042,0.428185,0.006379,...,1.191926,21.161911,-16.358565,11.592489,11.592489,1.138365,1.191926,0.029889,12.973622,2.204323


In [112]:
df = pd.DataFrame({'feature': fina_financial_ratios[fina_financial_ratios['years'].isin([2021])].Name, 'Return': prediction})
print(df.sort_values('Return', ascending=False))

     feature    Return
733     DXCM  0.859143
851     ENPH  0.818291
728      DVN  0.795775
866     EPAM  0.716986
1015    FTNT  0.689747
...      ...       ...
77       AFL -0.066428
627      COP -0.085233
861      EOG -0.086247
738     FANG -0.115506
669     CTRA -0.152823

[249 rows x 2 columns]


In [84]:
# Test
mae = mean_absolute_error(test_Y_metric*100, prediction*100)
mse = mean_squared_error(test_Y_metric*100, prediction*100, squared=False)

# Print
print("MAE: ", mae)
print("MSE: ", mse)

MAE:  37.423379930162305
MSE:  47.22435041267951


In [110]:
len(fina_financial_ratios.Name.unique())

245

In [85]:
importances = ran_f.feature_importances_

# To make it more understandable, let's create a DataFrame and print it
feature_importances = pd.DataFrame({'feature': train_X_metric.columns, 'importance': importances})
print(feature_importances.sort_values(by='importance', ascending=False))

                               feature  importance
0                                value    0.142471
1                          prev_return    0.122181
53             enterpriseValueMultiple    0.045847
50          priceEarningsToGrowthRatio    0.033413
5                            cashRatio    0.027146
18                          ebtPerEbit    0.026078
31                       assetTurnover    0.024561
52                       dividendYield    0.022099
14                      returnOnAssets    0.021585
47           priceToFreeCashFlowsRatio    0.021241
42                 dividendPayoutRatio    0.021107
39             shortTermCoverageRatios    0.020463
20                           debtRatio    0.020165
34                        cashPerShare    0.019357
46                  priceEarningsRatio    0.017931
35                         payoutRatio    0.015813
36         operatingCashFlowSalesRatio    0.015495
13                    effectiveTaxRate    0.015249
8            daysOfPayablesOuts

In [125]:
pd.DataFrame(
    {
        "Name": fina_financial_ratios.Name.unique(),
        "Prediction": prediction
    }
)

,Name,Prediction
0,MMM,0.074271
1,AOS,0.160337
2,ABT,0.247588
3,ABBV,0.116905
4,ACN,0.279427
...,...,...
240,HII,0.007006
241,IBM,0.246343
242,IEX,0.261154
243,IDXX,0.530136
